In [2]:
import numpy as np

# Sudoku 4x4

A 4x4 sudoku is composed of 4 "blocks". Each blocks contains a combination of numbers from 1 to 4 without repetition. So a single block alone can contain $4! = 24$ ways to organize those numbers.

If we begin with a single digit, let's say $1$. There's $3! = 6$ ways to organize the other numbers.


$$\begin{bmatrix}
1 & \_ \\
\_ & \_
\end{bmatrix}$$



In [15]:
b = np.array([[1,0],[0,0]])
n = 0

for i in range(1,5):
    if i != b[0,0] and i != b[1,1] and i != b[1,0]: 
        b[0,1] = i 
        for j in range(1,5):
            if j != b[0,0] and j != b[1,1] and j != b[0,1]:
                b[1,0] = j
                for k in range(1,5):
                    if k != b[0,0] and k != b[1,0] and k != b[0,1]:
                        b[1,1] = k
                        print(b.flatten(), end = " ")
                        b[1,1] = 0
                b[1,0] = 0
        b[0,1] = 0
    

[1 2 3 4] [1 2 4 3] [1 3 2 4] [1 3 4 2] [1 4 2 3] [1 4 3 2] 

For the other $18$ solutions of the single block, we only need to swap $1$ with any of the remaining allowed digits.

In [69]:
b = np.array([[0,0],[0,0]])
sol_b1 = []

for i in range(1, 5):
    b[0,0] = i
    for j in range(1,5):
        if j != b[0,0] and j != b[1,1] and j != b[1,0]: 
            b[0,1] = j 
            for k in range(1,5):
                if k != b[0,0] and k != b[1,1] and k != b[0,1]:
                    b[1,0] = k
                    for w in range(1,5):
                        if w != b[0,0] and w != b[1,0] and w != b[0,1]:
                            b[1,1] = w
                            sol_b1.append(b.copy())
                            # print(b.flatten(), end = " ")
                            b[1,1] = 0
                    b[1,0] = 0
            b[0,1] = 0
print("total solution = ", len(sol_b1))

total solution =  24


## Rules

$i : $ row index \
$j : $ column index \
$k : $ digit  

$C(i, j, k) : $ the cell located at row $i$ and column $j$ contains the digit $k$.

$\mathbf{V} :$ A valid sudoku 4x4 is filled with the numbers $1$ through $4$ so that each number appears exactly once in every row, column and block.

### Rule $B$

$B :$ Each cell of every block contains exactly one digit from the set $\{1,2,3,4\}$.

Our first rule can be defined as $$B \equiv \forall i\, \forall j\, \Big( \exists! k \in \{1,2,3,4\}, C(i, j, k)\Big),$$

meaning that for every cell $(i,j)$, there exists exactly one digit $k \in \{1,2,3,4\}$.

If we take the negation of $B$, we obtain $$\neg B \equiv \exists i\, \exists j\,
    \Big(
        \forall k \in \{1,2,3,4\}, \neg C(i,j,k)
        \lor
        \exists k \neq k' \in \{1,2,3,4\}, \big( C(i,j,k) \land C(i,j,k') \big)
    \Big)$$

Let's assume that $\neg B$ is true. Then there exists a cell $(i,j)$ that contains either no digit from the set $\{1,2,3,4\}$ or more than one digit from the set $\{1,2,3,4\}$. This contradicts the definition of a valid sudoku cell. Therefore, $B$ must be true.

let $x \in \{1,2,3,4\}$

Now let’s start by adding another block to the right of the previous one. We'll begin by imposing a single rule on the second block, the same as the first one : the block must be filled with each digit without repetition, so rule $B$.

$$
\begin{array}{c c}
\begin{bmatrix}
  \begin{bmatrix}
    \_ & \_ \\
    \_ & \_
  \end{bmatrix}
  &
  \begin{bmatrix}
    \_ & \_ \\
    \_ & \_
  \end{bmatrix}
\end{bmatrix}
\end{array}$$

This means that for each solutions of the first block, we have an additionnal $4! = 24$ solutions. So in total, we have $4! \times 4! = 576$ solutions.

To obtain these results, each solution of block 1 is joined with all other solutions of block 1.

In [74]:
sol_b2 = []

for s1 in sol_b1: 
    for s2 in sol_b1:
        sol = np.block([[s1, s2]])
        sol_b2.append(sol)
print("total solution = ", len(sol_b2))

total solution =  576


### Rule $I$
Now, let's continue by imposing our second rule: each digit must appear only once in every row.

We'll start with this as our baseline.

$$
\begin{array}{c c}
\begin{bmatrix}
  \begin{bmatrix}
    1 & 2 \\
    3 & 4
  \end{bmatrix}
  &
  \begin{bmatrix}
    \_ & \_ \\
    \_ & \_
  \end{bmatrix}
\end{bmatrix}
\end{array}$$

The rule begins with $\forall i [...]$ (for all row), so let us consider a fixed row to make it easier on us.

$$
\begin{array}{c c}
\begin{bmatrix}
  \begin{bmatrix}
    1 & 2 
  \end{bmatrix}
  &
  \begin{bmatrix}
    \_ & \_ 
  \end{bmatrix}
\end{bmatrix}
\end{array}$$

We can see that for row $i = 0$, in order to satisfy the given rule, every digit must appear exactly once across the columns.

Therefore, this gives $$\forall i\,  \forall k\, \exists! j \>\> C(i, j, k)$$

meaning that for every row $i$ and every digit $k$, there exists exactly one column $j$ where the cell $(i,j)$ contains $k$. Let's name this rule $I$

To validate our current sudoku, $I$ and $B$ must be true. Thus, the current rule set is $$I \land B. $$

If we take the negation of $I$, we get
$$\begin{aligned} 
\lnot I \equiv \exists i\, \exists k\, \Big( \forall j\,\lnot C(i, j, k)\, \lor\, \exists\,j_1 \neq j_2\,(C(i, j_1, k)\, \land\, C(i, j_2, k))\Big)
\end{aligned}$$

Now, let's assume that $\neg I$ is true, then there exists a row where at least one of the digits from the set $\{1,2,3,4\}$ is not present in any column or is present twice on the same row. 

Let $$P \equiv \forall j\,\lnot C(i, j, k), \quad Q \equiv \exists\,j_1 \neq j_2\,(C(i, j_1, k)\, \land\, C(i, j_2, k))$$

If $(P \lor Q)$ is true, this would violate the rule $\mathbf{V}$, since for $\mathbf{V}$ to hold, each digit from the set $\{1,2,3,4\}$ has to appear exactly once in every row, column and block. $$\neg I \implies \neg \mathbf{V}$$

Therefore, rule $I$ is necessary for a valid Sudoku.

In [67]:
sol_b2[0]

array([[1, 2, 3, 4],
       [1, 2, 3, 4]])

In [66]:
def valid_row(row):
    return len(row) == len(set(row))

grid = np.array([
    [1, 2, 3, 4],
    [3, 4, 1, 2]
])

for r in grid: print(valid_row(r))

True
True
